In [1]:
from bs4 import BeautifulSoup
import re
import pandas as pd
import csv 
import numpy as np
import os

In [67]:
def getFileName(originPath):
    ''' get all file name under originPath '''
    f_list_html = []
    f_list = os.listdir(originPath)
    # print f_list
    for i in f_list:
        # os.path.splitext():分离文件名与扩展名
        if os.path.splitext(i)[1] == '.html':
            f_list_html.append(os.path.splitext(i)[0])
    return f_list_html



In [68]:
originPath = 'data/'
#print(" f_list_html : ",getFileName(originPath))
print(" len  f_list_html : ",len(getFileName(originPath)))


 len  f_list_html :  226


In [69]:
#filename = '20150423_prop_for_sale_tom_classifieds'
#path = filename+'_Cleaned.csv'

In [70]:
#create csv file named path
def create_csv(path):

    with open(path, 'w', newline='') as csvfile:
        head = [e]
        writer = csv.DictWriter(csvfile, fieldnames=head)

        writer.writeheader()

In [71]:
#create_csv(path)

In [72]:
def write_csv(path,datarow):
    with open(path,'a') as f:
        writer = csv.writer(f, delimiter=',',)
        #writer.writerow({'Day_of_week': datarow[0], 'Month': datarow[1], 'Day_of_month': datarow[2],'Year': datarow[3],'City': datarow[4]})
        writer.writerow(datarow)

In [73]:
def geneRow_csv(HTMLfilename, outPath) : 
    soup = BeautifulSoup(open(HTMLfilename+".html"), 'html.parser')
    soup.prettify()




    #3.3.1 from day_of_week, month, day_of_month, year, city, description, price, phone
    '''we extract all except description and phone, because description is in natural language, so non structural data and 
    need NLP process, which we are not intented to do in this project for the moment; and the phone is not really useful for 
    statistical analyses that we do'''




    '''extract date'''
    
    for date in soup.find_all('h2', attrs={'class':'classified_date default_top_margin'}): 

        #print(date.get_text())
        reDateObj = re.compile('[, ]+')
        reDateObj.split(date.get_text())
        list_date_info = reDateObj.split(date.get_text())
        
        #data.append(list_date_info)
        #print(list_date_info)



        lengthAnnonce = len(date.next_sibling.next_sibling.find_all('p'))
        listAnnonce = date.next_sibling.next_sibling.find_all('p')
        for i in range(1, lengthAnnonce):
            #print("i : ", i)
            #print(listAnnonce[i].get_text())
            
             
            '''extract apartement'''
            
            Apartment = re.search(r"(?:[a-zA-Z]+[^a-zA-Z]*[,]*){1}apartment",listAnnonce[i].get_text())
            if(Apartment):
                apartment="y"
            else: 
                apartment="n"
            
            '''extract maisonette'''
                
            Maisonette = re.search(r"(?:[a-zA-Z]+[^a-zA-Z]*[,]*){1}maisonette",listAnnonce[i].get_text())
            if(Maisonette):
                maisonette="y"
            else: 
                maisonette="n"
            
            '''extract garage'''
            Garage = re.search(r"(?:[a-zA-Z]+[^a-zA-Z]*[,]*){1}garage",listAnnonce[i].get_text())
            if(Garage):
                garage="y"
            else: 
                garage="n"
            
            '''extract Pool'''
            Pool = re.search(r"(?:[a-zA-Z]+[^a-zA-Z]*[,]*){1}pool",listAnnonce[i].get_text())
            if(Pool):
                pool="y"
            else: 
                pool="n"
                
                
            '''extract garden'''
            Garden = re.search(r"(?:[a-zA-Z]+[^a-zA-Z]*[,]*){1}garden",listAnnonce[i].get_text())
            if(Garden):
                garden="y"
            else: 
                garden="n"

            '''extract city'''
            
      

            #reAnnonceObj = re.compile('[\n .]+')
            #newText = reAnnonceObj.split(listAnnonce[i].get_text())
            #print(newText)
            cities = re.findall('[A-ZĦŻĠÀĊ \'-]+\.|[A-ZĦŻĠÀĊ \'-]+\,|[A-ZĦŻĠÀĊ \'-]+\(', listAnnonce[i].get_text())
            #print("cities : ", cities)
            reDelim = re.compile('\.|\,|\(')
            city =''
            if (cities):
                cityWithOutDelim = reDelim.split(cities[0])
                city = cityWithOutDelim[0]
                #print("cityWithOutDelim[0] : ", cityWithOutDelim[0])
            else :
                city = 'NaN'




            '''extract price'''
            priceWithEuro = re.findall('€[0-9]+,[0-9]+', listAnnonce[i].get_text())
            #print("reEuro : ", priceWithEuro)

            reEuro = re.compile('[€]')

            if (priceWithEuro):
                priceWithOutEuro = reEuro.split(priceWithEuro[0])
                #print("priceWithOutEuro[1] : ", priceWithOutEuro[1])

                row = np.array([list_date_info[0], list_date_info[1], list_date_info[2], list_date_info[3], apartment, maisonette, garage, pool, garden, city, priceWithOutEuro[1]])


            else : 
                row = np.array([list_date_info[0], list_date_info[1], list_date_info[2], list_date_info[3], apartment, maisonette, garage, pool, garden, city, 'NaN'])
            list_r = row.tolist()

            write_csv(outPath,list_r)

            '''
            print("row :", row)
            print("len row :", len(row))
            print("type row :", type(row))
            print("list_row :", list_r)
            print("list_row :", type(list_r))
            print("len list_r", len(list_r))

            '''


In [74]:
def geneCSVforEachHTML(originPath):
    files_html = getFileName(originPath)
    for nameHTMLFile in files_html:
        outPutnameHTMLFile = 'dataCleaned/'+nameHTMLFile+'_Cleaned.csv'
        create_csv(outPutnameHTMLFile)
        print("originPath+nameHTMLFile+.csv", originPath+nameHTMLFile+".csv")
        
        geneRow_csv(originPath+nameHTMLFile, outPutnameHTMLFile)
        


In [75]:
geneCSVforEachHTML(originPath)   

originPath+nameHTMLFile+.csv data/20180827_prop_for_sale_tom_classifieds.csv
originPath+nameHTMLFile+.csv data/20180905_prop_for_sale_tom_classifieds.csv
originPath+nameHTMLFile+.csv data/20171227_prop_for_sale_tom_classifieds.csv
originPath+nameHTMLFile+.csv data/20150617_prop_for_sale_tom_classifieds.csv
originPath+nameHTMLFile+.csv data/20150928_prop_for_sale_tom_classifieds.csv
originPath+nameHTMLFile+.csv data/20160915_prop_for_sale_tom_classifieds.csv
originPath+nameHTMLFile+.csv data/20170124_prop_for_sale_tom_classifieds.csv
originPath+nameHTMLFile+.csv data/20160711_prop_for_sale_tom_classifieds.csv
originPath+nameHTMLFile+.csv data/20160229_prop_for_sale_tom_classifieds.csv
originPath+nameHTMLFile+.csv data/20200424_prop_for_sale_tom_classifieds.csv
originPath+nameHTMLFile+.csv data/20180409_prop_for_sale_tom_classifieds.csv
originPath+nameHTMLFile+.csv data/20160222_prop_for_sale_tom_classifieds.csv
originPath+nameHTMLFile+.csv data/20151214_prop_for_sale_tom_classifieds.csv

originPath+nameHTMLFile+.csv data/20200218_prop_for_sale_tom_classifieds.csv
originPath+nameHTMLFile+.csv data/20160822_prop_for_sale_tom_classifieds.csv
originPath+nameHTMLFile+.csv data/20170131_prop_for_sale_tom_classifieds.csv
originPath+nameHTMLFile+.csv data/20180820_prop_for_sale_tom_classifieds.csv
originPath+nameHTMLFile+.csv data/20181008_prop_for_sale_tom_classifieds.csv
originPath+nameHTMLFile+.csv data/20180312_prop_for_sale_tom_classifieds.csv
originPath+nameHTMLFile+.csv data/20180530_prop_for_sale_tom_classifieds.csv
originPath+nameHTMLFile+.csv data/20181003_prop_for_sale_tom_classifieds.csv
originPath+nameHTMLFile+.csv data/20150921_prop_for_sale_tom_classifieds.csv
originPath+nameHTMLFile+.csv data/20170704_prop_for_sale_tom_classifieds.csv
originPath+nameHTMLFile+.csv data/20180222_prop_for_sale_tom_classifieds.csv
originPath+nameHTMLFile+.csv data/20200615_prop_for_sale_tom_classifieds.csv
originPath+nameHTMLFile+.csv data/20161220_prop_for_sale_tom_classifieds.csv

originPath+nameHTMLFile+.csv data/20170319_prop_for_sale_tom_classifieds.csv
originPath+nameHTMLFile+.csv data/20170530_prop_for_sale_tom_classifieds.csv
originPath+nameHTMLFile+.csv data/20180919_prop_for_sale_tom_classifieds.csv
originPath+nameHTMLFile+.csv data/20200203_prop_for_sale_tom_classifieds.csv
originPath+nameHTMLFile+.csv data/20200511_prop_for_sale_tom_classifieds.csv
originPath+nameHTMLFile+.csv data/20180704_prop_for_sale_tom_classifieds.csv
originPath+nameHTMLFile+.csv data/20180822_prop_for_sale_tom_classifieds.csv
originPath+nameHTMLFile+.csv data/20180315_prop_for_sale_tom_classifieds.csv
originPath+nameHTMLFile+.csv data/20200211_prop_for_sale_tom_classifieds.csv
originPath+nameHTMLFile+.csv data/20150816_prop_for_sale_tom_classifieds.csv
originPath+nameHTMLFile+.csv data/20180829_prop_for_sale_tom_classifieds.csv
